In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN



In [ ]:
# 데이터 불러오기 - Apple 주식 데이터 (AAPL)
df = yf.download('AAPL', start='2010-01-01', end='2023-01-01')

# 종가 데이터 사용
df_close = df['Close']

# 데이터 전처리
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(np.array(df_close).reshape(-1,1))

# 학습 데이터 및 테스트 데이터 분리
training_size = int(len(scaled_data) * 0.7)
test_size = len(scaled_data) - training_size
train_data, test_data = scaled_data[0:training_size,:], scaled_data[training_size:len(scaled_data),:1]

# 데이터를 RNN에 입력할 수 있도록 시퀀스 데이터로 변환하는 함수
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

# 타임스텝 설정 (과거 60일치 데이터를 기반으로 예측)
time_step = 60
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# RNN 입력 차원 맞추기 (samples, time steps, features)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# RNN 모델 생성
model = Sequential()
model.add(SimpleRNN(50, return_sequences=True, input_shape=(time_step, 1)))
model.add(SimpleRNN(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)

# 예측 및 결과 시각화
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# 원래 스케일로 복원
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# 시각화
plt.plot(df_close[len(df_close)-len(test_predict)-1:].index, scaler.inverse_transform(scaled_data[len(train_data):]))
plt.plot(df_close[len(df_close)-len(test_predict)-1:].index, test_predict)
plt.title('Stock Price Prediction with RNN')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend(['Real Price', 'Predicted Price'], loc='best')
plt.show()